# Assignment 4 

In [1]:
import numpy as np
import sympy as sym
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import animatplot as amp

from numpy import exp
from textwrap import wrap
from matplotlib.animation import FuncAnimation
from scipy.constants import c, epsilon_0
from IPython.display import Math, display

from sympy import Derivative, symbols, simplify, fraction, Rational, I, sin, pi
from sympy.physics.mechanics import *
from IPython.display import Math, display

%config InlineBackend.figure_format = 'pdf'
init_vprinting()

In [2]:
N, q, m, C, eps0, mu0, omg, omg1, f1, gam1 = sym.symbols('N q m c \epsilon_0 \mu_0 \omega \omega_1 f_1 \gamma_1')

## 2) Absorbtion coefficient ($\alpha$) and index of refraction ($n$) in a conductor

Let: 

> $N$ : Molecules per unit volume

> $c$ : Speed of light

> $\epsilon_0$ : Permittivity of free space

> $q$ : Charge of the electron 

> $m$ : Electron mass

> $\omega$ = Frequency of the electric field

> $\omega_j$ = Frequency of the electron 

> $f_j$ : Number of electrons with frequency $\omega_j$

> $\gamma_j$ : Damping constant in each molecule


**Absorbtion coefficient:** 

$$\alpha = 2 \kappa \approx \frac{Nq^2 \omega^2}{m \epsilon_0 c} \sum_j \frac{f_j \gamma_j}{(\omega_j^2 - \omega^2)^2 + \gamma_j^2 \omega^2}$$


**Index of refraction:**

$$n = \frac{c k}{\omega} \approx 1 + \frac{Nq^2}{2 m \epsilon_0} \sum_j \frac{f_j (\omega_j^2 - \omega^2)}{(\omega_j^2 - \omega^2)^2 + \gamma_j^2 \omega^2}$$

In [3]:
# Code versions of the functions
# ==============================
def abs_coeff1(omega, N=1, q=1, omega_1=1, m=1, epsilon=1, c=1, f_1=1, gamma_1=0.1):
    A = N*(q**2)*(omega**2) / (m*epsilon*c)
    B = (f_1*gamma_1) / ((omega_1**2 - omega**2)**2 + (gamma_1**2) * (omega**2))
    alpha = A*B
    return alpha

def n_approx(omega, N=1, q=1, omega_1=1, m=1, epsilon=1, c=1, f_1=1, gamma_1=0.1):
    A = (N*q**2) / (2*m*epsilon)
    B = f_1 * (omega_1**2 - omega**2) / ((omega_1**2 - omega**2)**2 + (gamma_1**2) * (omega**2))
    n = 1 + A*B
    return n

### Absorbtion coefficient:

Let's look at the absorbtion coefficient first. 

For materials with a single natural frequency $\omega_1$:
$$\alpha \approx \frac{Nq^2 \omega^2}{m \epsilon_0 c} \frac{f_1 \gamma_1}{(\omega_1^2 - \omega^2)^2 + \gamma_1^2 \omega^2}$$

#### Finding $\omega$ for maximum $\alpha$:

By observation, we can see that $\alpha$ is maximum when $\omega = \omega_1$:
$$\alpha_{Max} \approx \frac{Nq^2}{m \epsilon_0 c} \frac{f_1 \gamma_1}{\gamma_1^2}$$ 

We can see that this is the case by setting everything to 1 and $\gamma_1 = 0.1$ (because we want $\alpha_{Max}$ to be finite), and plotting $\alpha$ with respect to $\omega$:

In [4]:
omega_space = np.arange(0.001, 2, 0.001)

fig = plt.figure(1, figsize=(10, 4))

plt.plot(omega_space, abs_coeff1(omega=omega_space, gamma_1=0.1), label=(r"$\alpha$ $\left[ \frac{q^2 \omega^2}{m \epsilon_0 c} \right]$"))
plt.plot(omega_space, n_approx(omega=omega_space, gamma_1=0.1), label=(r"$n$ $\left[ \frac{q^2}{m \epsilon_0 c} \right]$"))
plt.xlabel(r"$\omega$")
title1 = r"Relationship between absorbtion coefficient of a conductor ($\alpha$) and index of refraction ($n$), and incident frequency of light ($\omega$) with electron damping ($\gamma = 0.1$)"
plt.title("\n".join(wrap(title1, 100)))
plt.legend()
plt.grid()
plt.show()

<Figure size 720x288 with 1 Axes>

For a more rigorous proof, let's do this analytically:

Let's take the derivative of $\alpha$:

In [5]:
alpha_sym = abs_coeff1(omg, N, q, omg1, m, eps0, C, f1, gam1)
d_alpha_sym = sym.diff(alpha_sym, omg).simplify()
display(Math(r'\frac{\text{d} \alpha}{\text{d} \omega} =' + sym.latex(d_alpha_sym)))

<IPython.core.display.Math object>

Setting this to zero, we can obtain the values at which $\alpha$ is maximum:

In [6]:
omg_max = sym.solve(d_alpha_sym, omg)
display(Math(r'\omega_{Max} =' + sym.latex(omg_max)))

<IPython.core.display.Math object>

As frequency must be real and non-negative, we have $\omega = \omega_1$ which is thet same as what we observed. Substituting this into our equation for $\alpha$, this gives us an expression for $\alpha_{Max}$:

In [7]:
alpha_max = alpha_sym.subs(omg, omg1)
display(Math(r'\alpha_{Max} =' + sym.latex(alpha_max)))

<IPython.core.display.Math object>

#### Finding $\omega$ for $\frac{1}{2} \alpha_{Max}$:

To do this, we set $\alpha = \frac{1}{2} \alpha_{Max}$ then solve for $\omega$ 

In [8]:
omg_half = sym.solve(alpha_max/2 - alpha_sym, omg)
omg_half = [i.simplify() for i in omg_half]
display(Math(r'\omega_{1/2} =' + sym.latex(omg_half)))

<IPython.core.display.Math object>

We can see that the numerators are always positive.

Frequency can not be negative, so we can ignore two of these terms. We will have:

$$\omega_{1/2} = \pm \frac{\gamma_1}{2} + \frac{\sqrt{\gamma_1^2 + 4 \omega_1^2}}{2}$$

### Index of refraction:

#### Frequencies ($\omega$) for maximum and minimum index of refraction ($n$):

Take the derivative of $n$:

In [9]:
n_sym = n_approx(omg, N, q, omg1, m, eps0, C, f1, gam1)
d_n = sym.diff(n_sym, omg).simplify()
dd_n =  sym.diff(n_sym, omg, 2).simplify()
display(Math(r'n =' + sym.latex(n_sym)))
display(Math(r'\frac{\text{d} n}{\text{d} \omega} =' + sym.latex(d_n)))
display(Math(r'\frac{\text{d}^2 n}{\text{d}^2 \omega} =' + sym.latex(dd_n)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
omg_ex = sym.solve(d_n, omg)
display(Math(r'\omega_{Extremum} =' + sym.latex(omg_ex)))

<IPython.core.display.Math object>

Here's our values of $\omega$ for the extremums of $n$, but we don't know which ones are the minimum and which ones are the maximum. 

Let's substitute these values into $\frac{\text{d}^2 n}{\text{d}^2 \omega}$:

In [11]:
display(Math(r'\left.\frac{\text{d}^2 n}{\text{d}^2 \omega} \right|_{\omega_{Extremum}} =' + sym.latex([dd_n.subs(omg, omg_ex[i]).simplify() for i in range(len(omg_ex))])))

<IPython.core.display.Math object>

Assuming that $\omega_1 > \gamma_1$, (because we mostly deal with wavelengths shorter then infared) we can see:

> 1st term: Negative

> 2nd term: Negative

> 3rd term: Negative 

> 4th term: Positive

> 5th term: Positve 


This tells us that values of $\omega$ for maximum $n$: 

> $\omega_{Max_n} = \sqrt{\gamma_1 \omega_1 + \omega_1^2}$

Values of $\omega$ for minimum $n$: 

> $\omega_{Min_n} = \sqrt{-\gamma_1 \omega_1 + \omega_1^2}$

Here we have ignored $0$ and the negative terms of $\omega_{Extremum}$ because frequency must be positive. 

\newpage

## 3) Transverse electric waves in a rectangular waveguide

A rectangular waveguide with side lengths $0\leq x \leq a$ and $0 \leq y \leq b$.

The boundary conditions in a rectangular waveguide: 

$$\left\{ \begin{array}{ll}
    E^{\parallel} = 0\\
    B^{\perp} = 0
    \end{array} \right.$$  
    
The electric and magnetic fields have the generic form:

$$\left\{ \begin{array}{ll}
    E(x, y, z, t) = E_0(x, y) e^{i(kz-\omega t)}\\
    B(x, y, z, t) = B_0(x, y) e^{i(kz-\omega t)}
    \end{array} \right.$$ 
    
Following Griffiths, applying Maxwells equation to these and their results we (actually, just him) obtain the differential equations:

$$\left\{ \begin{array}{ll}
    \left[ \frac{\partial^2}{\partial^2 x} + \frac{\partial^2}{\partial^2 y} + \left(\frac{\omega}{c}\right)^2 - k^2 \right] E_z = 0\\
    \left[ \frac{\partial^2}{\partial^2 x} + \frac{\partial^2}{\partial^2 y} + \left(\frac{\omega}{c}\right)^2 - k^2 \right] B_z = 0
    \end{array} \right.$$ 

### a) Solving for TM waves:

To solve for TM waves, we set $B_z = 0$. 

Then we use separation of variables: 

$$E_z(x, y) = X(x)Y(y)$$

Substituting this into our differential equation for $E_z$ we obtain: 

$$Y \frac{\text{d}^2X}{\text{d}^2x} + X \frac{\text{d}^2Y}{\text{d}^2y} + \left[\left(\frac{\omega}{c}\right)^2 - k^2 \right] XY = 0$$

Dividing this by $XY$, we obtain: 

$$\left\{ \begin{array}{ll}
    \frac{1}{X} \frac{\text{d}^2X}{\text{d}^2x} = -k_x^2 \\
    \frac{1}{Y} \frac{\text{d}^2Y}{\text{d}^2y} = -k_y^2 \\
    -k_x^2 - k_y^2 + \left(\frac{\omega}{c}\right)^2 - k^2 = 0
    \end{array} \right.$$ 
    
We know the general solution (from math and more importantly, Griffiths) that the general solution: 

$$\left\{ \begin{array}{ll}
    X(x) = A \sin(k_x x) + B\cos(k_x x) \\
    Y(y) = D \sin(k_y y) + E\cos(k_y y)
    \end{array} \right.$$ 
    
    
The boundary conditions require $E^\parallel = 0$, therefore: 

$$\left\{ \begin{array}{ll}
    X(0) = A \sin(0) + B\cos(0) = 0 \\
    X(a) = A \sin(k_x a) + B\cos(k_x a) = 0 \\
    Y(0) = D \sin(0) + E\cos(0) = 0 \\
    Y(b) = D \sin(k_y m) + E\cos(k_y m) = 0
    \end{array} \right.$$ 

This gives us: 

$$\left\{ \begin{array}{ll}
    B = E = 0\\
    k_x = \frac{n \pi}{a} \\
    k_y = \frac{m \pi}{b}
    \end{array} \right.$$
    
$$\text{Where:  } (n, m) \in (0 < \mathbb{Z})$$

The equation to the TM wave:

$$\left\{ \begin{array}{ll}
    E_z(x, y) = E_0 \sin \left(\frac{n \pi}{a} x \right)\sin \left(\frac{m \pi}{b} y\right)\\
    k = \left[ \left(\frac{\omega}{c}\right)^2  - \left( \frac{n \pi}{a} \right)^2 - \left(\frac{m \pi}{b}\right)^2 \right]^{\frac{1}{2}}
    \end{array} \right.$$

### b) Poynting vector  and energy density for TM waves:

#### Poynting vector: 

$$\vec{S} = \frac{1}{\mu_0} Re\left( \vec{E} \times \vec{B^*} \right)$$


To find the Poynting vector, we need expressions for $E_x$ and $E_y$. Luckily, we have these relations, and know that $B_z = 0$:

$$\left\{ \begin{array}{ll}
    E_x = \frac{i}{(\omega/c)^2 - k^2}\left( k \frac{\partial E_z}{\partial x} + \omega \frac{\partial B_z}{\partial y} \right) = \frac{i k}{(\omega/c)^2 - k^2}\left( \frac{\partial E_z}{\partial x} \right)\\
    E_y = \frac{i}{(\omega/c)^2 - k^2}\left( k \frac{\partial E_z}{\partial y} - \omega \frac{\partial B_z}{\partial x} \right) = \frac{i k}{(\omega/c)^2 - k^2}\left( \frac{\partial E_z}{\partial y} \right)\\
    B_x = \frac{i}{(\omega/c)^2 - k^2}\left( k \frac{\partial B_z}{\partial x} - \frac{\omega}{c^2} \frac{\partial E_z}{\partial y} \right) = \frac{-i}{(\omega/c)^2 - k^2}\left( \frac{\omega}{c^2} \frac{\partial E_z}{\partial y} \right) \\
    B_y = \frac{i}{(\omega/c)^2 - k^2}\left( k \frac{\partial B_z}{\partial y} - \frac{\omega}{c^2} \frac{\partial E_z}{\partial x} \right) = \frac{i}{(\omega/c)^2 - k^2}\left( \frac{\omega}{c^2} \frac{\partial E_z}{\partial x} \right)
    \end{array} \right.$$
    
Let's make Sympy do this for us.

In [12]:
k, Ex, Ey, Ez, Bx, By, Bz, E0, B0 = sym.symbols('k E_x E_y E_z B_x B_y B_z E_0 B_0', complex=True) 
omg = sym.symbols('\omega', real=True)
a, b, x, y, C = sym.symbols('a b x y c', real=True, non_negative=True)
n, m = sym.symbols('n m', integer=True)

In [13]:
E_z = E0*sin((n*pi*x)/a)*sin((m*pi*y)/b)
d_Ez_x = sym.diff(E_z, x)
d_Ez_y = sym.diff(E_z, y)
display(Math(r'E_z =' + sym.latex(E_z)))
display(Math(r'\frac{\partial E_z}{\partial x} =' + sym.latex(d_Ez_x)))
display(Math(r'\frac{\partial E_z}{\partial y} =' + sym.latex(d_Ez_y)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
denom = (omg/C)**2 - k**2

E_x = I*k / denom * (d_Ez_x)
E_y = I*k / denom * (d_Ez_y)
B_x = -I*k / denom * (omg / C**2) * (d_Ez_y)
B_y = I*k / denom * (omg / C**2) * (d_Ez_x)

display(Math(r'E_x =' + sym.latex(E_x)))
display(Math(r'E_y =' + sym.latex(E_y)))
display(Math(r'B_x =' + sym.latex(B_x)))
display(Math(r'B_y =' + sym.latex(B_y)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
E = sym.Matrix([E_x, E_y, E_z])
B = sym.Matrix([B_x, B_y, 0])

Not_quite_S = 1/mu0 * (E.cross(sym.conjugate(B)))

Still_not_quite_S = sym.Matrix([Not_quite_S[0].simplify(), Not_quite_S[1].simplify(), Not_quite_S[2].simplify()])

display(Math(r'\frac{1}{\mu_0} \vec{E} \times \vec{B} = ' + sym.latex(Still_not_quite_S)))

<IPython.core.display.Math object>

The Poynting vector is $\vec{S} = \frac{1}{\mu_0} Re\left( \vec{E} \times \vec{B^*} \right)$:

In [16]:
display(Math(r'\vec{S} = S_z =' + sym.latex(Still_not_quite_S[2])))

<IPython.core.display.Math object>

We know that: 

$$\left\{ \begin{array}{ll}
    -k_x^2 - k_y^2 + \left(\frac{\omega}{c}\right)^2 - k^2 = 0\\
    k_x = \frac{n \pi}{a} \\
    k_y = \frac{m \pi}{b}
    \end{array} \right.$$

$$\therefore \omega^2 - (ck)^2 = \left( \frac{c n \pi}{a} \right)^2 + \left( \frac{ c m \pi}{b} \right)^2$$

Let's substitute this into our equation for Poynting vector then simplify. (We will do this automatically for future equations.)

In [17]:
Sub_a = (omg**2 - C**2 * k**2)*(omg**2 - C**2 * sym.conjugate(k)**2)
Sub_b = ((C*n*pi/a)**2 + (C*m*pi/b)**2)**2
Poynting = Still_not_quite_S[2].subs(Sub_a, Sub_b).simplify()
display(Math(r'\vec{S} = S_z =' + sym.latex(Poynting)))

<IPython.core.display.Math object>

Make this more readable: 

$$\vec{S} = \omega \epsilon_0 \left( \frac{|E_0 k| a b}{\pi (a^2 m^2 + b^2 n^2)} \right)^2 \left[a^2 m^2 \sin^2\left(\frac{n\pi x}{a}\right) \cos^2\left(\frac{m\pi x}{a}\right) + b^2 n^2 \sin^2\left(\frac{m\pi y}{b}\right) \cos^2\left(\frac{n\pi x}{a}\right)\right]\hat{z}$$

#### Energy density:

$$u = \frac{1}{2} \left( \epsilon_0 |\vec{E}|^2  + \frac{1}{\mu_0} |\vec{B}|^2 \right)$$

In [18]:
E_mag0 = (E.dot(sym.conjugate(E))).simplify()
B_mag0 = (B.dot(sym.conjugate(B))).simplify()
E_mag = E_mag0.subs(Sub_a, Sub_b).simplify()
B_mag = B_mag0.subs(Sub_a, Sub_b).simplify()

Energy_density = (Rational(1, 2) * (eps0 * E_mag + 1/mu0 * B_mag)).subs(Sub_a, Sub_b).simplify()
display(Math(r'|\vec{E}|^2 =' + sym.latex(E_mag)))
print('')
display(Math(r'|\vec{B}|^2 =' + sym.latex(B_mag)))
print('')
# display(Math(r'u =' + sym.latex(Energy_density)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

I'll just show $|\vec{E}|^2$ and $|\vec{B}|^2$, because $u$ is too long to print (I tried substituting them into $u$ then simplifying, and it still goes off page in landscape.)

Make this more eye-palatable:

$${\scriptstyle |\vec{E}|^2 = \left(\frac{|E_0|}{\pi c^2 (a^2 m^2 + b^2 n^2)}\right)^2 \left\{ (ab |k|)^2 \left[ a^2 m^2 \sin^2\left(\frac{n \pi x}{a}\right) \cos^2\left(\frac{m\pi y}{b}\right) + b^2 n^2 \sin^2\left(\frac{m\pi y}{b}\right)\cos^2\left(\frac{n\pi x}{a}\right)\right]+ \pi^2(a^2m^2 + b^2 n^2)^2\sin^2\left(\frac{n\pi x}{a}\right)\sin^2\left(\frac{m\pi y}{b}\right) \right\}}$$

$$|\vec{B}|^2 = \left(\frac{\omega a b |E_0|k}{c^2 \pi (a^2m^2 + b^2 n^2)^2}\right)^2 \left[ a^2 m^2 \sin^2\left(\frac{n \pi x}{a}\right) \cos^2\left(\frac{m \pi y}{b} \right) + b^2 n^2 \sin^2\left(\frac{m \pi y}{b}\right) \cos^2\left(\frac{n \pi x}{a} \right)\right]$$

### c) Integral of the Poynting vector and enegry density over cross sectional area of wave guide

#### Poynting vector:

In [19]:
Poynting_int = sym.integrate(Poynting, (x, 0, a), (y, 0, b)).simplify()
display(Math(r'\int \vec{S} \cdot \text{d}\vec{A} =' + sym.latex(Poynting_int)))

<IPython.core.display.Math object>

Make it nicer:

We can simplify this further by substituting equation (9.188), and equation (9.190):

$$\omega_{mn} = c \pi \sqrt{(m/a)^2 + (n/b)^2}$$

$$k = \frac{\sqrt{\omega^2 - \omega_{mn}^2}}{c}$$

$$\int \vec{S} \cdot \text{d} \vec{A} = \frac{\epsilon_0 \omega a^3 b^3 |E_0 k|^2}{4 \pi^2 (a^2 m^2 + b^2 n^2)} = \frac{\epsilon_0 \omega a b c^2 |E_0 k|^2}{4 \omega_{mn}^2} = \frac{\epsilon_0 \omega a b |E_0|^2 (\omega - \omega_{mn}^2)}{4 \omega_{mn}^2}$$

#### Enegry density:

In [20]:
Energy_int = sym.integrate(Energy_density, (x, 0, a), (y, 0, b)).simplify()
display(Math(r'\int u \ \text{d}A =' + sym.latex(Energy_int)))

<IPython.core.display.Math object>

Make this nicer as well: 

$$\int u \text{d} A = \frac{ab}{8 \mu_0} \left(\frac{|E_0|}{\pi c^2} \right)^2 \left[(ab|k|)^2 (\epsilon_0 \mu_0 + \omega^2) + (\epsilon_0 \mu_0 \pi c^2)^2 (a^2 m^2 + b^2 n^2) \right] = \frac{(ab)^3}{8 \mu_0} \left(\frac{|E_0|}{\pi c^2} \right)^2 \left[(|k|)^2 (c^2 + \omega^2) + \left(\frac{\omega_{mn}}{c}\right)^2 \right]$$

#### What do these represent?

The Poynting vector represents the energy flux, or energy per unit area per unit time. If we take its integral, we get the energy per unit time, or more appropriately, power. 

Energy density is the energy per unit volume, as the name suggests. If we take its integral, we get the energy per unit length. In our case, it is the energy per length of the wave guide (in the $\hat{z}$ direction). 

The ratio between the integral of the Poynting vector and the integral of the energy density will give us the speed at which energy is transmitted along the waveguide. In our case, it would be the speed at which energy is transmitted along the TM waveguide. 

$$v = \frac{\int \vec{S} \cdot \text{d} \vec{A}}{\int \vec{u} \ \text{d} A}$$

In [21]:
Energy_speed = (Poynting_int/Energy_int).simplify()
display(Math(r'v =' + sym.latex(Energy_speed)))

<IPython.core.display.Math object>

Make it look pretty:

$$v = \frac{2 \omega (abc|k|)^2}{(ab|k|)^2(\epsilon_0\mu_0c^4 + \omega^2) + \epsilon_0 \mu_0 \pi^2 c^4 (a^2 m^2 + b^2 n^2)} = \frac{2 \omega a^2 b^2 c^2 |k|^2}{a^2 b^2 |k|^2(c^2 + \omega^2) + \pi^2 c^2 (a^2 m^2 + b^2 n^2)}=  \frac{2 \omega a^2 b^2 c^2 |k|^2}{a^2 b^2 |k|^2(c^2 + \omega^2) + (ab \omega_{mn})^2} = \frac{2 \omega c^2 |k|^2}{|k|^2(c^2 + \omega^2) + \omega_{mn}^2}$$


We should get the equation for group velocicy, but something is wrong with my equations (probabily the integral for energy density), so I am unable to show it: 

$$v_g = c \sqrt{1-(\omega_{mn}/\omega)^2}$$

\newpage

## 4) COMPUTATION: Finite Difference Time Domain method

In [22]:
x_space = np.arange(-400, 401, 1)

In [23]:
t_max = 500
imp0 = 377
H_y = np.zeros((t_max, len(x_space)))
E_z = np.zeros((t_max, len(x_space)))
temp1 = np.zeros(len(x_space))
temp2 = np.zeros(len(x_space))

for i in range(0, t_max, 1):
    temp1[len(x_space)-1] = temp1[len(x_space)-2]
    for j in range(0, len(x_space)-1, 1):
        temp1[j] += (temp2[j+1] - temp2[j]) / imp0
    
    temp2[0] = temp2[1]
    for j in range(1, len(x_space), 1):    
        temp2[j] += (temp1[j] - temp1[j-1]) * imp0
    
    temp2[400] = exp(-(i-30)*(i-30)/100)
    
    E_z[i,:] = temp2[:]
    H_y[i,:] = temp1[:]

In [24]:
plt.figure(figsize = (10, 20))
for i in range(0, t_max, 10):
    plt.plot(x_space, E_z[i, :] + i/10, color="b", zorder=1, linewidth=1)
    plt.plot(x_space, H_y[i, :]*100 + i/10, color="orange", zorder=0, linewidth=1)
    
plt.legend((r'$|\vec{E}|$ (normalized)', r'$|\vec{B}|$ (relative to $|\vec{E}| \times 100$)'))
plt.xlabel('Space [spatial index]')
plt.ylabel('Time [frame number]')
plt.show()

<Figure size 720x1440 with 1 Axes>

In [28]:
%matplotlib notebook
# Animation
# =========
# def ani_func(i, E=E_z):
#     return E[:, i]
t = np.linspace(0, 0.001, 500)
timeline = amp.Timeline(t, units='s', fps=60)

fig = plt.figure()
block1 = amp.blocks.Line(E_z)
block2 = amp.blocks.Line(H_y*100)
anim = amp.Animation([block1, block2], timeline)

plt.ylim([-1.25,1.25])
# plt.tick_params(axis='both', which='both', 
#            bottom=False, top=False, left=False, right=False,
#            labelbottom=False, labeltop=False, labelleft=False, labelright=False,
#            labelsize=0)

anim.controls({'valfmt':'%.5f'})
#anim.timeline_slider(valfmt='%.5f')
#anim.toggle()
# anim.save_gif('ising')


#ani = FuncAnimation(fig, ani_func, frames=np.linspace(0, 200, 200), blit=True)
plt.show()

<IPython.core.display.Javascript object>